# Import libraries

In [19]:
import pandas as pd
import numpy as np
import os
import json
from os import listdir
import re
import sys
import ast
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
import string
import pickle
import datetime

nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/tie-server/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /home/tie-server/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/tie-
[nltk_data]     server/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# Change path to dataset here

In [20]:
basedir = os.path.join(os.sep, "media", "tie-server", "DATA", "Jens", "Crunchbase")
tempdir = os.getcwd()
#tempdir = basedir

# Import dataset

In [22]:
with open (os.path.join(basedir, 'temp_data', 'articles_df'), 'rb') as fp:
    articles_df = pickle.load(fp)

In [39]:
#articles_df.loc[:, "ids"] = [str(row["company"] + str(i)) for i, row in articles_df.iterrows()] 
articles_df.loc[:, "label"] = np.NaN
print(len(articles_df))
articles_df.head()

59535


,company,full_text,paragraphs,label
0,brightline,plus company updates(pcu)\n\n ...,new york: following is the company profile of...,NaN
1,brightline,plus company updates(pcu)\n\n ...,"""media agencies, creative agencies and publi...",NaN
2,brightline,m2 equitybites\n\n ...,brightline is the market leader for advanced t...,NaN
3,brightline,plus company updates(pcu)\n\n ...,austin: sizmek inc. has issued the following ...,NaN
4,brightline,broadcasting and cable\n\n ...,naylor said the majority of hulu's viewing now...,NaN


# Initialize a randomizer for our sample

In [42]:
import random
random.seed(130)
sample_ids = random.sample(list(articles_df.index), 200)

# The labelled data is having the following ids:

In [43]:
sample_ids_previous_sample = [5739, 949, 4425, 44820, 27899, 15983, 45289, 44438, 52763, 29693, 21203, 18728, 56765, 17490, 15750, 46752, 57532, 59063, 58417, 53887, 53811, 53189, 53039, 58913, 52134, 50769, 50755, 57612, 49925, 49813, 49812, 59025, 49811, 49205, 59321, 49170, 53938, 50198, 53999, 54637, 57648, 57672, 57388, 57190, 57050, 56856, 58155, 58280, 56052, 56051, 56050, 58300, 56049, 48174, 55875, 55102, 55069, 58416, 55036, 54372, 55876, 44686, 45365, 19863, 19671, 17916, 17915, 17889, 17848, 17653, 14639, 12766, 19957, 12620, 9962, 9503, 9502, 8868, 8830, 7904, 6222, 4722, 1951, 12202, 23894, 23930, 26410, 45197, 59441, 44159, 43445, 42518, 42061, 40398, 40397, 40002, 36959, 36895, 32870, 30814, 30224, 29501, 29245, 29244, 28891, 28773, 27145, 26604, 45758, 59526]
sample_ids = [i for i in sample_ids if i not in sample_ids_previous_sample]
print(len(sample_ids))

200


# Export sample for labelling

In [44]:
sample = articles_df[articles_df.index.isin(sample_ids)]
print(len(sample[sample.index.isin(sample_ids)]))
if not os.path.exists(os.path.join(tempdir, 'temp_data')):
    os.makedirs(os.path.join(tempdir, 'temp_data'))
sample.to_csv(path_or_buf=os.path.join(os.getcwd(), 'temp_data', 'sample4label.csv'), columns=["company", "paragraphs", "label"], sep=";",index=True, header=True)
sample.head()

200


,company,full_text,paragraphs,label
202,crono,daily news (south africa)\n\n ...,cronos: guillermo del toro's surreal directori...,NaN
1445,motopia,rwe company announcements\n\n ...,* motopia ltd has announced the appointment o...,NaN
1505,upsnap,market news publishing\n\n ...,"upsnap inc. (""upsnap"" or the ""company""), is pl...",NaN
1891,solarreserve,mining magazine\n\n ...,"link explains: ""the energy-supply agreement ca...",NaN
1893,solarreserve,fierceenergy\n\n m...,"""we are pleased to continue our efforts to bri...",NaN


## Take your time to label data... 

# Load the labelled sample 
The ids of the labelled sample is matched to the ones from the dataset

In [26]:
articles_df1 = articles_df.copy()

sample_labelled_df = pd.read_csv(
    filepath_or_buffer=os.path.join(tempdir, 'temp_data', 'sample4label_labelled.csv'), 
    sep=";", index_col=0)


#sample_labelled_df = sample_labelled_df[sample_labelled_df.index.isin(sample_ids)]
sample_labelled_df.reset_index(inplace=True)
sample_labelled_df.set_index(keys=["paragraphs"], inplace=True)
articles_df1.set_index(keys=["paragraphs"], inplace=True)
articles_df1.update(sample_labelled_df["label"])
articles_df1.reset_index(inplace=True)
sample_labelled_df = articles_df1[~articles_df1["label"].isnull()].sort_values(by="label")
print(len(sample_labelled_df))
#print(list(sample_labelled_df.index))

110
[5739, 949, 4425, 44820, 27899, 15983, 45289, 44438, 52763, 29693, 21203, 18728, 56765, 17490, 15750, 46752, 57532, 59063, 58417, 53887, 53811, 53189, 53039, 58913, 52134, 50769, 50755, 57612, 49925, 49813, 49812, 59025, 49811, 49205, 59321, 49170, 53938, 50198, 53999, 54637, 57648, 57672, 57388, 57190, 57050, 56856, 58155, 58280, 56052, 56051, 56050, 58300, 56049, 48174, 55875, 55102, 55069, 58416, 55036, 54372, 55876, 44686, 45365, 19863, 19671, 17916, 17915, 17889, 17848, 17653, 14639, 12766, 19957, 12620, 9962, 9503, 9502, 8868, 8830, 7904, 6222, 4722, 1951, 12202, 23894, 23930, 26410, 45197, 59441, 44159, 43445, 42518, 42061, 40398, 40397, 40002, 36959, 36895, 32870, 30814, 30224, 29501, 29245, 29244, 28891, 28773, 27145, 26604, 45758, 59526]


In [25]:
sample_labelled_df.head()

,paragraphs,company,full_text,label
5739,"the ""protection fee"" demanded bitcoin payments...",cloudflare,sc magazine (us)\n\n ...,N
949,"""we have seen tremendous growth in our m&a bus...",fleetmatics,"pehub\n\n september 6, 201...",P
4425,devops automates processes between development...,virtustream,plus company updates(pcu)\n\n ...,P
44820,cardiokinetix has raised us $44 million in a ...,cardiokinetix,cardiovascular device business\n\n ...,P
27899,the biggest change is found in the browser's...,ghostery,indian technology news\n\n ...,P


In [13]:
#print(json.dumps(list(articles_df1[articles_df1["label"] == "P"]["content"]), indent=4))
print(str(len(articles_df1[(~articles_df1["label"].isnull()) & (articles_df.duplicated(subset='full_text', keep=False))].sort_values(by="label"))) + " duplicated articles in the sample")

print(len(sample_labelled_df))

0 duplicated articles in the sample
110


/home/tie-server/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


# Print out the questionable articles

In [14]:
#print(json.dumps(list(sample_labelled_df[sample_labelled_df["label"] == "Q"]["content"]), sort_keys=True, indent=4))

# Get rid of questionable articles and convert labels to meaningful

In [15]:
sample_labelled_df = sample_labelled_df[~(sample_labelled_df["label"] == "Q")]
sample_labelled_df.replace(to_replace={"label": "N"}, value="negative", inplace=True)
sample_labelled_df.replace(to_replace={"label": "X"}, value="neutral", inplace=True)
sample_labelled_df.replace(to_replace={"label": "P"}, value="positive", inplace=True)

len(sample_labelled_df)


98

# Save the sample

In [18]:
if not os.path.exists(os.path.join(basedir, 'temp_data')):
    os.makedirs(os.path.join(basedir, 'temp_data'))
with open(os.path.join(basedir, 'temp_data', 'sample_labelled_df'), 'wb') as fp:
    pickle.dump(sample_labelled_df, fp)